In [1]:
import pandas as pd
from yelpapi import YelpAPI

# 1. Process columns


In [1]:
df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/data/stockholm_housing_df_RAW2.csv")

df['fee'] = df['fee'].str.rstrip(' kr/mån')

#Drop 'location' column, as it is redundant
df = df.drop(columns=["location"])

#Split size of apartment and number of rooms into separate columns
df[['size', 'rooms']] = df['size:rooms'].str.split("  ", 1, expand=True)
df = df.drop(columns=["size:rooms"])
#Clean up the columns size and room, removing m2 and "rum"
df['size'] = df['size'].str.rstrip('m²')
df["rooms"] = df["rooms"].str.rstrip(' rum')

#Split adress and floor of apt into separate columns
df[['adress', 'floor']] = df['adress'].str.split(",", 1, expand=True)

#Clean up sold_date, sale_price, value_dev, ppsqm
df["sold_date"] = df["sold_date"].str.lstrip('såld')
df["sale_price"] = df["sale_price"].str.lstrip('slutpris ')
df["sale_price"] = df["sale_price"].str.rstrip(' kr')
df["ppsqm"] = df["ppsqm"].str.rstrip(" kr/m²")
df["value_dev"] = df["value_dev"].str.rstrip(" %")
df["value_dev"] = df["value_dev"].str.lstrip("+")
df["value_dev"] = df["value_dev"].str.lstrip("±")
df["value_dev"].astype(float)

# Introduce dummy variables in the following way: Balkong = 1, Hiss = 2, Balkong&Hiss = 3, Uteplats = 4
df["features"] = df.features.map( {'balkong':1 , 'hiss':2, 'balkong&hiss':3, 'uteplats': 4, 'NaN':5} )

#Remove everything except numerical values from floor
df['floor'] = df['floor'].str.extract('(\d+)', expand=False)

#Bug where value_dev ends up in wrong columns, due to old postings
bad_index = df[df["ppsqm"].str.find("%") != -1].index
df = df.drop(bad_index, axis=0)

column_list = ["ppsqm", "size", "rooms", "sale_price","fee"]

def fix_numeric(column_list, df):
    new_df = df.copy()
    for i in column_list: 
        new_df[i] = new_df[i].str.replace(" ", "")
        new_df[i] = pd.to_numeric(new_df[i] , errors='coerce')
        new_df = new_df.dropna(subset=[i])
        new_df[i] = new_df[i].astype('int')
    return new_df

cleaned_housing_df = fix_numeric(column_list,df.copy() )
#Reset index after removing rows
cleaned_housing_df = cleaned_housing_df.reset_index(drop=True)
cleaned_housing_df.to_csv("stockholm_housing_df_CLEANED2.csv",index=False)


NameError: name 'pd' is not defined

# Yelp API calls

In [6]:
yelp_df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/data/stockholm_housing_df_CLEANED2.csv")


In [ ]:
yelp_api = YelpAPI("KulP_1xAbhj4PcwcltixYR5hz4qMJ2aarTp4uNP_bBED4CsgP1nqY0bZrDxRMMSsZYwqvirOQ1Dy--6v3Y2yS4lBVPmfebDVdXGukr74OZEKRNoivTBiORBJ0v_iYXYx")

#A 'suggested search area' of 500 is used, although, the docs tell us the following: 
#This field is used as a suggestion to the search. The actual search radius may be lower 
#than the suggested radius in dense urban areas, and higher in regions of less business density
#hopefully, it does give some quantifiable value as to the 'closeness' to points of interest a apartment has

k = 0
def get_POI(adress):
    input_adress = adress
    response = yelp_api.search_query(location=input_adress, radius=500, limit=1)
    return response

def get_yelp_values(df, k):
    try:
        for i in range(k, len(df)):
            response = get_POI(df["adress"][i])
            print("Getting values from adress:", df["adress"][i], "from row: " , i)
            
            lat = ( response['region']['center']['latitude'] )
            long = ( response['region']['center']['longitude'] )
            POI = ( response['total'])
            df.at[i, 'Latitude'] = lat
            df.at[i, 'Longitude'] = long
            df.at[i, 'NearbyPOIs'] = POI
            
        return df
    except Exception as e:
        print(e)
        print("\n","error")
        k = i+1
        
        error_handler(df,k)
        
        
def error_handler(df,k):
    get_yelp_values(df,k)
    return 

get_yelp_values(yelp_df,k)

yelp_df.to_csv("UNFINISHED_yelp_df_CLEANED.csv",index=False)

Getting values from adress: tegnérlunden 6 from row:  0
Getting values from adress: sveavägen 93 from row:  1
Getting values from adress: norrbackagatan 27a from row:  2
Getting values from adress: tomtebogatan 30b from row:  3
Getting values from adress: vanadisvägen 39 from row:  4
Getting values from adress: kammakargatan 64 from row:  5
Getting values from adress: norrbackagatan 42 from row:  6
Getting values from adress: gästrikegatan 8 from row:  7
Getting values from adress: falugatan 17 from row:  8
Getting values from adress: sveavägen 117a from row:  9
Getting values from adress: gävlegatan 32 from row:  10
Getting values from adress: sankt eriksgatan 106 from row:  11
Getting values from adress: norrbackagatan 14 from row:  12
Getting values from adress: tomtebogatan 3 from row:  13
Getting values from adress: david bagares gata 12a from row:  14
Getting values from adress: idungatan 1b from row:  15
Getting values from adress: rörstrandsgatan 40 from row:  16
Getting values

Getting values from adress: fleminggatan 60 c from row:  138
Getting values from adress: storgatan 32 from row:  139
Getting values from adress: linnégatan 39a from row:  140
Getting values from adress: kommendörsgatan 24 from row:  141
Getting values from adress: styrmansgatan 37 from row:  142
Getting values from adress: riddargatan 29 c from row:  143
Getting values from adress: sibyllegatan 50 b from row:  144
Getting values from adress: styrmansgatan 7a from row:  145
Getting values from adress: odengatan 6a from row:  146
Getting values from adress: kaptensgatan 11 from row:  147
Getting values from adress: kommendörsgatan 42 from row:  148
Getting values from adress: brahegatan 23 from row:  149
Getting values from adress: brahegatan 58 from row:  150
Getting values from adress: sibyllegatan 29 from row:  151
Getting values from adress: gyllenstiernsgatan 17 from row:  152
Getting values from adress: artillerigatan 63 from row:  153
Getting values from adress: surbrunnsgatan 9 f

Getting values from adress: anders reimers väg 11 from row:  275
Getting values from adress: skaraborgsgatan 6a from row:  276
Getting values from adress: fredmansgatan 7 from row:  277
Getting values from adress: marmorgatan 11 from row:  278
Getting values from adress: ringvägen 158 from row:  279
Getting values from adress: åsögatan 184 from row:  280
Getting values from adress: tullgårdsgatan 22 from row:  281
Getting values from adress: slipgatan 12 from row:  282
Getting values from adress: högalidsgatan 11 from row:  283
Getting values from adress: kocksgatan 51 from row:  284
Getting values from adress: samaritgränd 1 from row:  285
Getting values from adress: drakenbergsgatan 11 from row:  286
Getting values from adress: åsögatan 96 from row:  287
Getting values from adress: åsögatan 184 from row:  288
Getting values from adress: östgötagatan 36 from row:  289
Getting values from adress: metargatan 13 from row:  290
Getting values from adress: lundagatan 38a from row:  291
Get